In [2]:
import rospy
from geometry_msgs.msg import Twist
import sys, select, os
if os.name == 'nt':
    import msvcrt
else:
    import tty, termios

MAX_LIN_VEL = 0.40
MAX_ANG_VEL = 2.79

# burger : 0.22/2.84    waffle : 0.26/1.82

LIN_VEL_STEP_SIZE = 0.01
ANG_VEL_STEP_SIZE = 0.1


In [3]:
def getKey():
    if os.name == 'nt':
        return msvcrt.getch()

    tty.setraw(sys.stdin.fileno())
    rlist, _, _ = select.select([sys.stdin], [], [], 0.1)
    if rlist:
        key = sys.stdin.read(1)
    else:
        key = ''

    termios.tcsetattr(sys.stdin, termios.TCSADRAIN, settings)
    return key

In [4]:
def vels(target_linear_x_vel, target_linear_y_vel, target_angular_vel):
    return "currently:\tlinear_vel\tx %s\ty %s\t angular vel %s " % (target_linear_x_vel,target_linear_y_vel,target_angular_vel)

In [5]:
def makeSimpleProfile(output, input, slop):
    if input > output:
        output = min( input, output + slop )
    elif input < output:
        output = max( input, output - slop )
    else:
        output = input

    return output

In [6]:
def constrain(input, low, high):
    if input < low:
        input = low
    elif input > high:
        input = high
    else:
        input = input

    return input

In [7]:
def checkLinearLimitVelocity(vel):
    vel = constrain(vel, -MAX_LIN_VEL, MAX_LIN_VEL)

    return vel

In [8]:
def checkAngularLimitVelocity(vel):
    vel = constrain(vel, -MAX_ANG_VEL, MAX_ANG_VEL)

    return vel

In [24]:
rospy.init_node('turtlebot3_teleop')
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

# Function to Move

In [33]:
def turtlebotMove(action, time_move=0.5):
    # Variables
    control_linear_x_vel = 0   
    target_linear_x_vel = checkLinearLimitVelocity(action)
    
    # To control the board
    twist = Twist()
    control_linear_x_vel = makeSimpleProfile(control_linear_x_vel, target_linear_x_vel, time_move)
    twist.linear.x = control_linear_x_vel; 
    twist.linear.y = 0.0; twist.linear.z = 0.0
    twist.angular.x = 0.0; twist.angular.y = 0.0; twist.angular.z = 0.0
    
    # Move!
    pub.publish(twist)

# Function to Rotate

In [ ]:
def turtlebotRotate(action, time_rotate=0.2):
    # Variables
    control_angular_vel  = 0.0
    target_angular_vel = checkAngularLimitVelocity(action)

    # To control the board
    twist = Twist()
    twist.linear.x = 0.0; twist.linear.y = 0.0; twist.linear.z = 0.0
    control_angular_vel = makeSimpleProfile(control_angular_vel, target_angular_vel, time_rotate)
    twist.angular.x = 0.0; twist.angular.y = 0.0; twist.angular.z = control_angular_vel
    
    # Rotate!
    pub.publish(twist)